In [4]:
!pip install faiss-cpu sentence-transformers openai


In [6]:
import faiss
import numpy as np
from sentence_transformers import SentenceTransformer
import openai


/opt/anaconda3/lib/python3.12/site-packages/sentence_transformers/cross_encoder/CrossEncoder.py:13: TqdmExperimentalWarning: Using `tqdm.autonotebook.tqdm` in notebook mode. Use `tqdm.tqdm` instead to force console mode (e.g. in jupyter console)
  from tqdm.autonotebook import tqdm, trange


In [8]:
!pip install ipywidgets
!jupyter nbextension enable --py widgetsnbextension


usage: jupyter [-h] [--version] [--config-dir] [--data-dir] [--runtime-dir]
               [--paths] [--json] [--debug]
               [subcommand]

Jupyter: Interactive Computing

positional arguments:
  subcommand     the subcommand to launch

options:
  -h, --help     show this help message and exit
  --version      show the versions of core jupyter packages and exit
  --config-dir   show Jupyter config dir
  --data-dir     show Jupyter data dir
  --runtime-dir  show Jupyter runtime dir
  --paths        show all Jupyter paths. Add --json for machine-readable
                 format.
  --json         output paths as machine-readable json
  --debug        output debug information about paths

Available subcommands: console dejavu events execute kernel kernelspec lab
labextension labhub migrate nbconvert notebook qtconsole run server
troubleshoot trust

Jupyter command `jupyter-nbextension` not found.


In [ ]:
!conda install -c conda-forge ipywidgets


Retrieving notices: ...working... done
Channels:
 - conda-forge
 - defaults
Platform: osx-arm64
Solving environment: done

## Package Plan ##

  environment location: /opt/anaconda3

  added / updated specs:
    - ipywidgets


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    ipywidgets-8.1.5           |     pyhd8ed1ab_0         111 KB  conda-forge
    jupyterlab_widgets-3.0.13  |     pyhd8ed1ab_0         182 KB  conda-forge
    pydeck-0.8.0               |     pyhd8ed1ab_0         3.9 MB  conda-forge
    widgetsnbextension-4.0.13  |     pyhd8ed1ab_0         878 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         5.0 MB

The following packages will be UPDATED:

  ipywidgets         pkgs/main/osx-arm64::ipywidgets-7.8.1~ --> conda-forge/noarch::ipywidgets-8.1.5-pyhd8ed1ab_0 
  jupyterlab_widgets pkgs/main

In [10]:
# Load a pre-trained model
model = SentenceTransformer('all-MiniLM-L6-v2')  # Or another suitable model


In [16]:
def create_faiss_index(docs):
    # Encode documents to embeddings and move them to the CPU
    embeddings = model.encode(docs, convert_to_tensor=True).cpu().detach().numpy()
    
    # Initialize FAISS index
    index = faiss.IndexFlatL2(embeddings.shape[1])  # L2 distance metric
    index.add(embeddings)
    return index, embeddings


In [20]:
# Example document chunks for each city
city_data = {
    "Jacksonville": ["Document chunk 1 for Jacksonville", "Document chunk 2 for Jacksonville"],
    "Miami": ["Document chunk 1 for Miami", "Document chunk 2 for Miami"],
    # Add other cities here
}

# Initialize dictionaries to store indexes and embeddings for each city
city_indexes = {}
city_embeddings = {}

# Loop through city_data and create FAISS indexes for each city
for city, docs in city_data.items():
    index, embeddings = create_faiss_index(docs)
    city_indexes[city] = index
    city_embeddings[city] = embeddings


In [22]:
def retrieve_from_all_cities(query, city_indexes, top_k=3):
    query_embedding = model.encode([query], convert_to_tensor=True).cpu().detach().numpy()
    results = {}

    for city, index in city_indexes.items():
        _, top_indices = index.search(query_embedding, top_k)
        top_docs = [city_data[city][idx] for idx in top_indices[0]]
        results[city] = top_docs

    return results


In [24]:
def query_llm_combined_results(query, city_indexes):
    retrieved_docs = retrieve_from_all_cities(query, city_indexes)
    
    # Combine document chunks from all cities
    combined_docs = "\n\n".join([f"{city}:\n" + "\n".join(docs) for city, docs in retrieved_docs.items()])

    # Create the prompt for the LLM
    prompt = f"Answer the question based on the following city-specific climate action plans:\n\n{combined_docs}\n\nQuestion: {query}"
    
    # Query the LLM (OpenAI in this case)
    response = openai.Completion.create(
        engine="text-davinci-003",  # Use the appropriate model
        prompt=prompt,
        max_tokens=300,
        temperature=0.7
    )
    return response.choices[0].text.strip()


In [30]:

import openai

# Set your OpenAI API key
openai.api_key = "sk-proj-gxZhQ2sAl0TmsQzDA2qOT3BlbkFJA0AZTS59Eo0GiScnuJoc"

query = "How are cities addressing coastal flooding in their climate action plans?"

def query_llm_combined_results(query, city_indexes):
    retrieved_docs = retrieve_from_all_cities(query, city_indexes)
    
    # Combine document chunks from all cities
    combined_docs = "\n\n".join([f"{city}:\n" + "\n".join(docs) for city, docs in retrieved_docs.items()])

    # Create the LLM prompt
    prompt = f"Answer the question based on the following city-specific climate action plans:\n\n{combined_docs}\n\nQuestion: {query}"
    
    # Query the LLM with gpt-3.5-turbo
    response = openai.ChatCompletion.create(
        model="gpt-3.5-turbo",
        messages=[
            {"role": "system", "content": "You are an assistant that helps synthesize city climate action plans."},
            {"role": "user", "content": prompt}
        ],
        max_tokens=300,
        temperature=0.7
    )
    
    return response['choices'][0]['message']['content']

response = query_llm_combined_results(query, city_indexes)
print("LLM Response:", response)


LLM Response: In Jacksonville's climate action plan, there is a focus on coastal flooding mitigation through measures such as enhancing green infrastructure, implementing stormwater management systems, and promoting coastal resiliency strategies. Specifically, the plan outlines the importance of protecting coastal areas from flooding and sea-level rise by incorporating nature-based solutions and sustainable infrastructure to reduce vulnerability.

Miami's climate action plan also addresses coastal flooding by prioritizing adaptation strategies to protect against the impacts of rising sea levels and extreme weather events. The plan emphasizes the need for improving coastal infrastructure, implementing flood management programs, and enhancing coastal resiliency measures to safeguard communities from the risks associated with coastal flooding. Additionally, Miami's plan highlights the importance of collaboration with stakeholders and leveraging innovative technologies to address the chall